<a href="https://colab.research.google.com/github/Jaroslav789/Data_analysis_Colab/blob/main/Stock_Dip_Analyzer_FUNGUJE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import os
import numpy as np

# ==============================================================================
# Funkce pro výpočet ATR s EMA vyhlazováním (Wilder's Smoothing)
# Přebírá logiku z MRZ skeneru pro konzistenci
# ==============================================================================
def calculate_atr(high, low, close, period=5):
    """
    Calculate Average True Range (ATR) using Exponential Moving Average (EMA)
    which is standard (Wilder's smoothing) for trading platforms.
    """
    # 1. High - Low
    high_low = high - low
    # 2. Absolute value of High - Previous Close
    high_close = np.abs(high - close.shift())
    # 3. Absolute value of Low - Previous Close
    low_close = np.abs(low - close.shift())

    # True Range (TR) is the maximum of the three ranges
    true_range = np.maximum.reduce([high_low.values, high_close.values, low_close.values])
    true_range_series = pd.Series(true_range, index=close.index)

    # Calculate ATR using EWM (Exponential Weighted Moving Average) for Wilder's Smoothing
    # adjust=False ensures the calculation aligns with standard ATR definitions (like AmiBroker)
    atr = true_range_series.ewm(span=period, adjust=False).mean()
    return atr


# ==============================================================================
# 1) Ruční seznam tickerů Nasdaq-100
# ==============================================================================
tickers = [
"NVDA","AAPL","MSFT","AMZN","GOOGL","GOOG","AVGO","META","TSLA","NFLX",
"ASML","PLTR","COST","AMD","CSCO","AZN","MU","TMUS","APP","AMAT",
"SHOP","INTC","ISRG","PEP","LRCX","LIN","QCOM","AMGN","INTU","PDD",
"BKNG","TXN","KLAC","GILD","ARM","ADBE","PANW","ADI","CRWD","HON",
"CEG","VRTX","MELI","ADP","CMCSA","SBUX","DASH","CDNS","ORLY","SNPS",
"MAR","MRVL","REGN","CTAS","MNST","ABNB","MDLZ","ADSK","CSX","AEP",
"FTNT","WBD","PYPL","TRI","IDXX","ROST","WDAY","DDOG","PCAR","NXPI",
"MSTR","EA","BKR","ROP","XEL","FAST","TTWO","EXC","FANG","AXON",
"CCEP","TEAM","PAYX","ZS","KDP","CPRT","CTSH","GEHC","VRSK","MCHP",
"KHC","ODFL","CSGP","BIIB","DXCM","CHTR","LULU","ON","GFS","TTD","CDW"
]


print(f"Loading {len(tickers)} tickers...")

# ==============================================================================
# 2) Stažení dat (210 dní pro výpočet MA200)
# ==============================================================================
# yf.download stáhne historické ceny pro všechny tickery najednou.
data = yf.download(tickers, period="210d", auto_adjust=True)

# ==============================================================================
# 3) Výpočet výsledků
# ==============================================================================
results = []

for t in tickers:

    # Vyber data pro ticker a zahoď řádky, kde chybí data
    try:
        # Používá .xs pro výběr sloupce s daným tickerem (na úrovni 1)
        df = data[['Open', 'High', 'Low', 'Close']].xs(t, level=1, axis=1).dropna()
    except KeyError:
        continue # Pokud ticker není dostupný, přeskočíme

    if len(df) < 200:
        # Potřebujeme dostatek dat pro spolehlivý výpočet MA200
        continue

    # Zjištění cen
    yesterday_close = df['Close'].iloc[-2]
    today_close     = df['Close'].iloc[-1]

    # % změna ceny dnešního close oproti včerejšímu close
    pct_change = (today_close - yesterday_close) / yesterday_close * 100

    # ========================================
    # 🎯 VÝPOČET ATR(5) s EMA (Wilder's Smoothing)
    # ========================================
    # Používá sdílenou funkci calculate_atr pro ATR(5)
    atr5 = calculate_atr(df['High'], df['Low'], df['Close'], period=5).iloc[-1]
    # ========================================

    # MA200 (200-denní klouzavý průměr)
    ma200 = df['Close'].rolling(200).mean().iloc[-1]
    above_ma200 = today_close > ma200

    # Vypočet vstupních a výstupních limitů na základě ATR(5)
    # Vstup (Entry Limit / Stop Loss): Dnešní Close - 0.9 * ATR(5)
    entry_limit = today_close - 0.9 * atr5
    # Výstup (Profit Target / Take Profit): Dnešní Close + 0.5 * ATR(5)
    exit_limit  = today_close + 0.5 * atr5


    # --- Filtr: akcie musela poklesnout o 3% a musí být nad MA200 ---
    if pct_change <= -3 and above_ma200:
        results.append({
            "Ticker": t,
            "YesterdayClose": round(yesterday_close, 2),
            "Close": round(today_close, 2),
            "PctChange": round(pct_change, 2),
            "ATR5": round(atr5, 2),
            "Price": round(entry_limit, 2),
            "ProfitTarget": round(exit_limit, 2),
            "AboveMA200": above_ma200
        })

# ==============================================================================
# 4) Zpracování výstupu
# ==============================================================================
results_df = pd.DataFrame(results)

# Přidání sloupce Příkaz = "Buy"
results_df["Příkaz"] = "Buy"

print("\n=== RESULTS ===")

if results_df.empty:
    print("No stocks meet criteria (PctChange ≤ -3% & Above MA200).")
else:
    # Vytvoření cesty a názvu souboru pro uložení
    datum = datetime.now().strftime("%Y%m%d")
    nazev_souboru = f"{datum}_dip.csv"
    cesta = r"C:\Users\Administrator\Documents\trading_01\dip_system\csv_files"

    # Zajištění existence adresáře
    os.makedirs(cesta, exist_ok=True)
    full_path = os.path.join(cesta, nazev_souboru)

    # Uložení do CSV
    results_df.to_csv(full_path, index=False, encoding="utf-8-sig")
    print(f"Soubor byl uložen jako: {full_path}")

    # Zobrazení výsledků (simulace display(results_df) pro prostředí Canvas)
    print("\nVyfiltrované obchodní signály:")
    print(results_df.to_string())

Loading 101 tickers...


[*********************100%***********************]  101 of 101 completed



=== RESULTS ===
Soubor byl uložen jako: C:\Users\Administrator\Documents\trading_01\dip_system\csv_files/20251205_dip.csv

Vyfiltrované obchodní signály:
  Ticker  YesterdayClose   Close  PctChange   ATR5   Price  ProfitTarget  AboveMA200 Příkaz
0     MU          234.16  226.65      -3.21  11.13  216.63        232.21        True    Buy
1   INTC           43.76   40.50      -7.45   2.65   38.12         41.82        True    Buy
2    MAR          306.65  296.00      -3.47   8.56  288.30        300.28        True    Buy
3     ON           57.15   54.79      -4.13   3.23   51.88         56.41        True    Buy


In [6]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import os
import numpy as np

# ==============================================================================
# Funkce pro výpočet ATR s EMA vyhlazováním (Wilder's Smoothing)
# UPRAVENO: Explicitní výpočet prvního ATR jako SMA pro lepší shodu s AmiBroker.
# ==============================================================================
def calculate_atr(high, low, close, period=5):
    """
    Calculate Average True Range (ATR) using the standard (Wilder's) smoothing method.
    The first ATR value is calculated as a Simple Moving Average (SMA) of True Range,
    and subsequent values use the EMA smoothing formula for high compatibility
    with trading platforms like AmiBroker.
    """
    # 1. True Range Calculation
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())

    true_range = np.maximum.reduce([high_low.values, high_close.values, low_close.values])
    true_range_series = pd.Series(true_range, index=close.index)

    # Kontrola, zda máme dostatek dat pro výpočet
    if len(true_range_series) <= period:
        return true_range_series.fillna(0) # Vrátí nuly nebo NaN, pokud není dost dat

    # 2. ATR Calculation (Wilder's Smoothing s SMA inicializací)
    atr_values = [np.nan] * len(true_range_series)

    # Index, kde začíná první plnohodnotné ATR (perioda + 1 den)
    start_index = period

    # Krok A: Nastav SMA True Range pro prvních 'period' TR jako první plné ATR.
    # TR má NaN první den, takže bereme indexy 1 až period.
    atr_values[start_index] = true_range_series.iloc[1:period+1].mean()

    # Krok B: Aplikuj Wilder's smoothing na zbytek
    # Wilder's formula: ATR_i = (ATR_{i-1} * (period - 1) + TR_i) / period
    for i in range(start_index + 1, len(true_range_series)):
        atr_values[i] = (atr_values[i-1] * (period - 1) + true_range_series.iloc[i]) / period

    return pd.Series(atr_values, index=close.index)


# ==============================================================================
# 1) Ruční seznam tickerů Nasdaq-100
# ==============================================================================
tickers = [
"NVDA","AAPL","MSFT","AMZN","GOOGL","GOOG","AVGO","META","TSLA","NFLX",
"ASML","PLTR","COST","AMD","CSCO","AZN","MU","TMUS","APP","AMAT",
"SHOP","INTC","ISRG","PEP","LRCX","LIN","QCOM","AMGN","INTU","PDD",
"BKNG","TXN","KLAC","GILD","ARM","ADBE","PANW","ADI","CRWD","HON",
"CEG","VRTX","MELI","ADP","CMCSA","SBUX","DASH","CDNS","ORLY","SNPS",
"MAR","MRVL","REGN","CTAS","MNST","ABNB","MDLZ","ADSK","CSX","AEP",
"FTNT","WBD","PYPL","TRI","IDXX","ROST","WDAY","DDOG","PCAR","NXPI",
"MSTR","EA","BKR","ROP","XEL","FAST","TTWO","EXC","FANG","AXON",
"CCEP","TEAM","PAYX","ZS","KDP","CPRT","CTSH","GEHC","VRSK","MCHP",
"KHC","ODFL","CSGP","BIIB","DXCM","CHTR","LULU","ON","GFS","TTD","CDW"
]


# ==============================================================================
# 2) Stažení dat (210 dní pro výpočet MA200)
# ==============================================================================
# yf.download stáhne historické ceny pro všechny tickery najednou.
data = yf.download(tickers, period="210d", auto_adjust=True)

# ==============================================================================
# 3) Výpočet výsledků
# ==============================================================================
results = []

for t in tickers:

    # Vyber data pro ticker a zahoď řádky, kde chybí data
    try:
        # Používá .xs pro výběr sloupce s daným tickerem (na úrovni 1)
        df = data[['Open', 'High', 'Low', 'Close']].xs(t, level=1, axis=1).dropna()
    except KeyError:
        continue # Pokud ticker není dostupný, přeskočíme

    if len(df) < 200:
        # Potřebujeme dostatek dat pro spolehlivý výpočet MA200
        continue

    # Zjištění cen
    yesterday_close = df['Close'].iloc[-2]
    today_close     = df['Close'].iloc[-1]

    # % změna ceny dnešního close oproti včerejšímu close
    pct_change = (today_close - yesterday_close) / yesterday_close * 100

    # ========================================
    # 🎯 VÝPOČET ATR(5) s EMA (Wilder's Smoothing)
    # ========================================
    # Používá sdílenou funkci calculate_atr pro ATR(5)
    # Tato funkce nyní explicitně inicializuje ATR pomocí SMA pro lepší AmiBroker shodu.
    atr5 = calculate_atr(df['High'], df['Low'], df['Close'], period=5).iloc[-1]
    # ========================================

    # MA200 (200-denní klouzavý průměr)
    ma200 = df['Close'].rolling(200).mean().iloc[-1]
    above_ma200 = today_close > ma200

    # Vypočet vstupních a výstupních limitů na základě ATR(5)
    # Vstup (Entry Limit / Stop Loss): Dnešní Close - 0.9 * ATR(5)
    entry_limit = today_close - 0.9 * atr5
    # Výstup (Profit Target / Take Profit): Dnešní Close + 0.5 * ATR(5)
    exit_limit  = today_close + 0.5 * atr5


    # --- Filtr: akcie musela poklesnout o 3% a musí být nad MA200 ---
    if pct_change <= -3 and above_ma200:
        results.append({
            "Ticker": t,
            "YesterdayClose": round(yesterday_close, 2),
            "Close": round(today_close, 2),
            "PctChange": round(pct_change, 2),
            "ATR5": round(atr5, 2),
            "Price": round(entry_limit, 2),
            "ProfitTarget": round(exit_limit, 2),
            "AboveMA200": above_ma200
        })

# ==============================================================================
# 4) Zpracování výstupu
# ==============================================================================
results_df = pd.DataFrame(results)

# Přidání sloupce Příkaz = "Buy"
results_df["Příkaz"] = "Buy"

print("\n=== RESULTS ===")

if results_df.empty:
    print("No stocks meet criteria (PctChange ≤ -3% & Above MA200).")
else:
    # Vytvoření cesty a názvu souboru pro uložení
    datum = datetime.now().strftime("%Y%m%d")
    nazev_souboru = f"{datum}_dip.csv"
    cesta = r"C:\Users\Administrator\Documents\trading_01\dip_system\csv_files"

    # Zajištění existence adresáře
    os.makedirs(cesta, exist_ok=True)
    full_path = os.path.join(cesta, nazev_souboru)

    # Uložení do CSV
    results_df.to_csv(full_path, index=False, encoding="utf-8-sig")
    print(f"Soubor byl uložen jako: {full_path}")

    # Zobrazení výsledků (simulace display(results_df) pro prostředí Canvas)
    print("\nVyfiltrované obchodní signály:")
    print(results_df.to_string())

[*********************100%***********************]  101 of 101 completed



=== RESULTS ===
Soubor byl uložen jako: C:\Users\Administrator\Documents\trading_01\dip_system\csv_files/20251205_dip.csv

Vyfiltrované obchodní signály:
  Ticker  YesterdayClose   Close  PctChange   ATR5   Price  ProfitTarget  AboveMA200 Příkaz
0     MU          234.16  226.65      -3.21  12.31  215.57        232.81        True    Buy
1   INTC           43.76   40.50      -7.45   2.40   38.34         41.70        True    Buy
2    MAR          306.65  296.00      -3.47   7.62  289.14        299.81        True    Buy
3     ON           57.15   54.79      -4.13   2.85   52.23         56.21        True    Buy
